# Predicting Draft Picks

Now that I have all my data, I need to combine my two dataframes and start cleaning. I load in my necessary libraries and read in the two csv files saved off in my web scraping notebook.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# setting format for readability
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [5]:
combine = pd.read_csv('2000-2020_nfl_combine_data.csv', index_col = 0)
combine.head()

,Link,PlayerName,Position,School,CollegeStats,Height,Weight,40yd,Vertical,Bench,BroadJump,3Cone,Shuttle,Drafted,CombineYear,Draft Team,Round,Pick,Draft Year
0,https://www.pro-football-reference.com/draft/2...,John Abraham,OLB,South Carolina,/td,6-4,252,4.55,nan,nan,nan,nan,nan,New York Jets / 1st / 13th pick / 2000,2000,New York Jets,1st,13th pick,2000
1,https://www.pro-football-reference.com/draft/2...,Shaun Alexander,RB,Alabama,https://www.sports-reference.com/cfb/players/s...,6-0,218,4.58,nan,nan,nan,nan,nan,Seattle Seahawks / 1st / 19th pick / 2000,2000,Seattle Seahawks,1st,19th pick,2000
2,https://www.pro-football-reference.com/draft/2...,Darnell Alford,OT,Boston Col.,/td,6-4,334,5.56,25.00,23.00,94.00,8.48,4.98,Kansas City Chiefs / 6th / 188th pick / 2000,2000,Kansas City Chiefs,6th,188th pick,2000
3,https://www.pro-football-reference.com/draft/2...,Kyle Allamon,TE,Texas Tech,/td,6-2,253,4.97,29.00,nan,104.00,7.29,4.49,NaN,2000,NaN,Not Drafted,Not Drafted,Not Drafted
4,https://www.pro-football-reference.com/draft/2...,Rashard Anderson,CB,Jackson State,/td,6-2,206,4.55,34.00,nan,123.00,7.18,4.15,Carolina Panthers / 1st / 23rd pick / 2000,2000,Carolina Panthers,1st,23rd pick,2000


In [6]:
collegestats = pd.read_csv('college_stats_final.csv', index_col = 0)
collegestats.head()

,Link,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,Rushing/Receiving_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,#Punts,Punt_Yds,Yds_per_Punt,PuntRet_Games,Punt_Returns,Punt_Return_Yds,Yrds_per_Return,Punt_Returned_for_TD,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Scoring_Games,TD_Other,Ttl_TDs,2PT_Converstions,Safety,TTL_Points
0,https://www.sports-reference.com/cfb/players/s...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327,1706,5.20,23,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,nan,24.00,nan,nan,144
1,https://www.sports-reference.com/cfb/players/l...,12.00,nan,nan,nan,nan,nan,1.00,27.00,27.00,0.00,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,nan,0.00,nan,nan,0
2,https://www.sports-reference.com/cfb/players/j...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,1.00,1.00,100.00,27.00,27.00,27.00,0.00,0.00,326.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
3,https://www.sports-reference.com/cfb/players/a...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,nan,5.00,nan,nan,30
4,https://www.sports-reference.com/cfb/players/t...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,180.00,295.00,61.00,2217.00,7.50,7.70,16.00,6.00,138.00,11.00,34.00,-31.00,-0.90,1.00,nan,nan,nan,nan,34,-31,-0.90,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,nan,1.00,nan,nan,6


I decided to rename my 'Link' column in my collegestats dataframe. This is the column I'll merge my two dataframes on and having the same name keeps things clear for me. The name doesn't have to be the same, but again it's just for clarity on my end.

In [7]:
collegestats.rename({'Link': 'CollegeStats'}, axis=1, inplace=True)
collegestats.head()

,CollegeStats,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,Rushing/Receiving_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,#Punts,Punt_Yds,Yds_per_Punt,PuntRet_Games,Punt_Returns,Punt_Return_Yds,Yrds_per_Return,Punt_Returned_for_TD,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Scoring_Games,TD_Other,Ttl_TDs,2PT_Converstions,Safety,TTL_Points
0,https://www.sports-reference.com/cfb/players/s...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327,1706,5.20,23,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,nan,24.00,nan,nan,144
1,https://www.sports-reference.com/cfb/players/l...,12.00,nan,nan,nan,nan,nan,1.00,27.00,27.00,0.00,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,nan,0.00,nan,nan,0
2,https://www.sports-reference.com/cfb/players/j...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,1.00,1.00,100.00,27.00,27.00,27.00,0.00,0.00,326.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
3,https://www.sports-reference.com/cfb/players/a...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,nan,5.00,nan,nan,30
4,https://www.sports-reference.com/cfb/players/t...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,180.00,295.00,61.00,2217.00,7.50,7.70,16.00,6.00,138.00,11.00,34.00,-31.00,-0.90,1.00,nan,nan,nan,nan,34,-31,-0.90,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,nan,1.00,nan,nan,6


In [8]:
df = pd.merge(combine, collegestats, how='left', left_on='CollegeStats', right_on='CollegeStats', )

# check out the newly merged dataframe
print(df.shape)
df.head()

(6899, 83)


,Link,PlayerName,Position,School,CollegeStats,Height,Weight,40yd,Vertical,Bench,BroadJump,3Cone,Shuttle,Drafted,CombineYear,Draft Team,Round,Pick,Draft Year,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,Rushing/Receiving_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,#Punts,Punt_Yds,Yds_per_Punt,PuntRet_Games,Punt_Returns,Punt_Return_Yds,Yrds_per_Return,Punt_Returned_for_TD,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Scoring_Games,TD_Other,Ttl_TDs,2PT_Converstions,Safety,TTL_Points
0,https://www.pro-football-reference.com/draft/2...,John Abraham,OLB,South Carolina,/td,6-4,252,4.55,nan,nan,nan,nan,nan,New York Jets / 1st / 13th pick / 2000,2000,New York Jets,1st,13th pick,2000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,https://www.pro-football-reference.com/draft/2...,Shaun Alexander,RB,Alabama,https://www.sports-reference.com/cfb/players/s...,6-0,218,4.58,nan,nan,nan,nan,nan,Seattle Seahawks / 1st / 19th pick / 2000,2000,Seattle Seahawks,1st,19th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327.00,1706.00,5.20,23.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,nan,24.00,nan,nan,144.00
2,https://www.pro-football-reference.com/draft/2...,Darnell Alford,OT,Boston Col.,/td,6-4,334,5.56,25.00,23.00,94.00,8.48,4.98,Kansas City Chiefs / 6th / 188th pick / 2000,2000,Kansas City Chiefs,6th,188th pick,2000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,https://www.pro-football-reference.com/draft/2...,Kyle Allamon,TE,Texas Tech,/td,6-2,253,4.97,29.00,nan,104.00,7.29,4.49,NaN,2000,NaN,Not Drafted,Not Drafted,Not Drafted,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,https://www.pro-football-reference.com/draft/2...,Rashard Anderson,CB,Jackson State,/td,6-2,206,4.55,34.00,nan,123.00,7.18,4.15,Carolina Panthers / 1st / 23rd pick / 2000,2000,Carolina Panthers,1st,23rd pick,2000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [9]:
# pickle combined data to keep it stored

df.to_pickle('combined_data')

Not all of my original data points had the college stats available. From here on, I'll only want to look at the observations that I was able to get the additional information on. I also go ahead and drop the 'Link', 'CollegeStats' and 'Drafted' columns. I've taken what I need from these columns and they don't need to be included from here on out.

In [10]:
draft_data = df[df['CollegeStats'] != '/td']
draft_data.drop(columns=['Link', 'CollegeStats', 'Drafted'], inplace=True)
print(draft_data.shape)
draft_data.head()

(5492, 80)


,PlayerName,Position,School,Height,Weight,40yd,Vertical,Bench,BroadJump,3Cone,Shuttle,CombineYear,Draft Team,Round,Pick,Draft Year,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,Rushing/Receiving_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,#Punts,Punt_Yds,Yds_per_Punt,PuntRet_Games,Punt_Returns,Punt_Return_Yds,Yrds_per_Return,Punt_Returned_for_TD,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Scoring_Games,TD_Other,Ttl_TDs,2PT_Converstions,Safety,TTL_Points
1,Shaun Alexander,RB,Alabama,6-0,218,4.58,nan,nan,nan,nan,nan,2000,Seattle Seahawks,1st,19th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327.00,1706.00,5.20,23.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,nan,24.00,nan,nan,144.00
6,LaVar Arrington,OLB,Penn State,6-3,250,4.53,nan,nan,nan,nan,nan,2000,Washington Redskins,1st,2nd pick,2000,12.00,nan,nan,nan,nan,nan,1.00,27.00,27.00,0.00,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,nan,0.00,nan,nan,0.00
10,John Baker,P,North Texas,6-3,227,nan,nan,nan,nan,nan,nan,2000,NaN,Not Drafted,Not Drafted,Not Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,1.00,1.00,100.00,27.00,27.00,27.00,0.00,0.00,326.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
18,Anthony Becht,TE,West Virginia,6-6,270,4.78,33.50,nan,123.00,6.94,4.08,2000,New York Jets,1st,27th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,nan,5.00,nan,nan,30.00
27,Tom Brady,QB,Michigan,6-4,211,5.28,24.50,nan,99.00,7.20,4.38,2000,New England Patriots,6th,199th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,180.00,295.00,61.00,2217.00,7.50,7.70,16.00,6.00,138.00,11.00,34.00,-31.00,-0.90,1.00,nan,nan,nan,nan,34.00,-31.00,-0.90,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,nan,1.00,nan,nan,6.00


The way 'Height' came in isn't really functional for analysis. I want to convert the values to inches to then see distributions. I split the original 'Height' column into two - 'Feet' & 'Inches'. For 'Feet', I need to multiply the separated value by 12 and then I can add the two together. This value will replace 'Height' - renamed 'Height(in)' - and I can drop 'Feet' and 'Inches' when ready.

In [11]:
draft_data['Feet'] = draft_data['Height']
draft_data['Feet'] = draft_data['Feet'].map(lambda x: int((x.split('-')[0])) * 12)
draft_data['Feet'].head()

1     72
6     72
10    72
18    72
27    72
Name: Feet, dtype: int64

In [12]:
draft_data['Inches'] = draft_data['Height']
draft_data['Inches'] = draft_data['Inches'].map(lambda x: int(x.split('-')[1]))
draft_data['Inches'].head()

1     0
6     3
10    3
18    6
27    4
Name: Inches, dtype: int64

In [13]:
draft_data['Height'] = draft_data['Feet'] + draft_data['Inches']
draft_data.rename({'Height': 'Height(in)'}, axis=1, inplace=True)
draft_data['Height(in)'].head()

1     72
6     75
10    75
18    78
27    76
Name: Height(in), dtype: int64

In [14]:
# whoops! misspelt Conversion - quick rename!

draft_data.rename({'2PT_Converstions': '2PT_Conversions'}, axis=1, inplace=True)

I have a ton of NA's in my data set, but they're not you're typical null value. The nature of the different positions means that there will understandably be nulls. You would never expect a kicker to have an interception. I've decided to fill all null value with 0 for later assessment. I also reorganized some of my columns to better keep track of what I'm looking at.

In [15]:
draft_data.fillna(0, inplace=True)

In [16]:
draft_data = draft_data[['PlayerName', 'Position', 'School', 'CombineYear','Height(in)', 'Weight', '40yd',
       'Vertical', 'Bench', 'BroadJump', '3Cone', 'Shuttle',
       'Draft Team', 'Round', 'Pick', 'Draft Year', 'Defense_Games',
       'Solo_Tackles', 'Assisted_Tackles', 'Ttl_Tackles', 'Loss', 'Sacks',
       'Defensive_Interceptions', 'Def_Int_Yds', 'Yds_per_Int', 'Pick_6',
       'Defended_Passes', 'Recovered_Fumbles', 'Rec_Fumbles_Yds',
       'Fumbles_Returned_TD', 'Forced_Fumbles', 'Passing_Games', 'Completions',
       'Pass_Attempts', 'Completion_Percent', 'Pass_Yards',
       'Pass_Yds_per_Attempt', 'Adj_Pass_Yds_per_Attempt', 'Pass_TDs',
       'Pass_Interceptions', 'Passer_Rating', 'Rushing/Receiving_Games',
       'Rush_Attempts', 'Rush_Yds', 'Rush_Yds_per_Attempt', 'Rush_TDs',
       'Receptions', 'Rec_Yds', 'Rec_Yds_per_Reception', 'Rec_TDs',
       'Plays_from_Scrimmage', 'Scrimmage_Yds', 'Scrimmage_Yds_per_Attempt',
       'Scrimmage_TDs', 'Kicking_Games', 'XP_Made', 'XP_Attempts',
       'XP_Percent', 'FG_Made', 'FG_Attempts', 'FG_Percent',
       'TTL_Kicking_Points', '#Punts', 'Punt_Yds', 'Yds_per_Punt',
       'PuntRet_Games', 'Punt_Returns', 'Punt_Return_Yds', 'Yrds_per_Return',
       'Punt_Returned_for_TD', 'Kickoff_Returns', 'KO_Return_Yds',
       'Yds_per_KO_Return', 'KO_Returned_for_TD', 'Scoring_Games', 'TD_Other',
       'Ttl_TDs', '2PT_Conversions', 'Safety', 'TTL_Points', 'Feet',
       'Inches']]

draft_data.head()

,PlayerName,Position,School,CombineYear,Height(in),Weight,40yd,Vertical,Bench,BroadJump,3Cone,Shuttle,Draft Team,Round,Pick,Draft Year,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,Rushing/Receiving_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,#Punts,Punt_Yds,Yds_per_Punt,PuntRet_Games,Punt_Returns,Punt_Return_Yds,Yrds_per_Return,Punt_Returned_for_TD,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Scoring_Games,TD_Other,Ttl_TDs,2PT_Conversions,Safety,TTL_Points,Feet,Inches
1,Shaun Alexander,RB,Alabama,2000,72,218,4.58,0.00,0.00,0.00,0.00,0.00,Seattle Seahawks,1st,19th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327.00,1706.00,5.20,23.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,24.00,0.00,0.00,144.00,72,0
6,LaVar Arrington,OLB,Penn State,2000,75,250,4.53,0.00,0.00,0.00,0.00,0.00,Washington Redskins,1st,2nd pick,2000,12.00,0.00,0.00,0.00,0.00,0.00,1.00,27.00,27.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,0.00,0.00,0.00,0.00,0.00,72,3
10,John Baker,P,North Texas,2000,75,227,0.00,0.00,0.00,0.00,0.00,0.00,0,Not Drafted,Not Drafted,Not Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,1.00,1.00,100.00,27.00,27.00,27.00,0.00,0.00,326.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72,3
18,Anthony Becht,TE,West Virginia,2000,78,270,4.78,33.50,0.00,123.00,6.94,4.08,New York Jets,1st,27th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,5.00,0.00,0.00,30.00,72,6
27,Tom Brady,QB,Michigan,2000,76,211,5.28,24.50,0.00,99.00,7.20,4.38,New England Patriots,6th,199th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,180.00,295.00,61.00,2217.00,7.50,7.70,16.00,6.00,138.00,11.00,34.00,-31.00,-0.90,1.00,0.00,0.00,0.00,0.00,34.00,-31.00,-0.90,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,6.00,72,4


In [17]:
def per_game_data(column_list, game_column):
    for i in column_list:
        metric = i
        column_name = metric + '/Game'
#         print(column_name)
        draft_data[column_name] = draft_data[i]/draft_data[game_column]
           

In [18]:
defense = ['Solo_Tackles', 'Assisted_Tackles', 'Ttl_Tackles', 'Loss', 'Sacks', 'Defensive_Interceptions', 
           'Def_Int_Yds', 'Pick_6', 'Defended_Passes', 'Recovered_Fumbles', 'Rec_Fumbles_Yds',
           'Fumbles_Returned_TD', 'Forced_Fumbles']

passing = ['Completions', 'Pass_Attempts', 'Pass_Yards', 'Pass_TDs', 'Pass_Interceptions']

rushing = ['Rush_Attempts', 'Rush_Yds', 'Rush_TDs', 'Receptions', 'Rec_Yds', 'Rec_TDs', 'Plays_from_Scrimmage',
           'Scrimmage_Yds', 'Scrimmage_TDs']

kicking = ['XP_Made', 'XP_Attempts', 'FG_Made', 'FG_Attempts', 'TTL_Kicking_Points', '#Punts', 'Punt_Yds']

punt_ret = ['Punt_Returns', 'Punt_Return_Yds', 'Punt_Returned_for_TD', 'Kickoff_Returns', 'KO_Return_Yds', 
            'KO_Returned_for_TD']

scoring = ['TD_Other', 'Ttl_TDs', '2PT_Conversions', 'Safety', 'TTL_Points']

In [19]:
per_game_data(defense, 'Defense_Games')
per_game_data(passing, 'Passing_Games')
per_game_data(rushing, 'Rushing/Receiving_Games')
per_game_data(kicking, 'Kicking_Games')
per_game_data(punt_ret, 'PuntRet_Games')
per_game_data(scoring, 'Scoring_Games')

In [20]:
draft_data.columns

Index(['PlayerName', 'Position', 'School', 'CombineYear', 'Height(in)',
       'Weight', '40yd', 'Vertical', 'Bench', 'BroadJump',
       ...
       'Punt_Return_Yds/Game', 'Punt_Returned_for_TD/Game',
       'Kickoff_Returns/Game', 'KO_Return_Yds/Game', 'KO_Returned_for_TD/Game',
       'TD_Other/Game', 'Ttl_TDs/Game', '2PT_Conversions/Game', 'Safety/Game',
       'TTL_Points/Game'],
      dtype='object', length=127)

In [21]:
draft_data.head()

,PlayerName,Position,School,CombineYear,Height(in),Weight,40yd,Vertical,Bench,BroadJump,3Cone,Shuttle,Draft Team,Round,Pick,Draft Year,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,Rushing/Receiving_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,#Punts,Punt_Yds,Yds_per_Punt,PuntRet_Games,Punt_Returns,Punt_Return_Yds,Yrds_per_Return,Punt_Returned_for_TD,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Scoring_Games,TD_Other,Ttl_TDs,2PT_Conversions,Safety,TTL_Points,Feet,Inches,Solo_Tackles/Game,Assisted_Tackles/Game,Ttl_Tackles/Game,Loss/Game,Sacks/Game,Defensive_Interceptions/Game,Def_Int_Yds/Game,Pick_6/Game,Defended_Passes/Game,Recovered_Fumbles/Game,Rec_Fumbles_Yds/Game,Fumbles_Returned_TD/Game,Forced_Fumbles/Game,Completions/Game,Pass_Attempts/Game,Pass_Yards/Game,Pass_TDs/Game,Pass_Interceptions/Game,Rush_Attempts/Game,Rush_Yds/Game,Rush_TDs/Game,Receptions/Game,Rec_Yds/Game,Rec_TDs/Game,Plays_from_Scrimmage/Game,Scrimmage_Yds/Game,Scrimmage_TDs/Game,XP_Made/Game,XP_Attempts/Game,FG_Made/Game,FG_Attempts/Game,TTL_Kicking_Points/Game,#Punts/Game,Punt_Yds/Game,Punt_Returns/Game,Punt_Return_Yds/Game,Punt_Returned_for_TD/Game,Kickoff_Returns/Game,KO_Return_Yds/Game,KO_Returned_for_TD/Game,TD_Other/Game,Ttl_TDs/Game,2PT_Conversions/Game,Safety/Game,TTL_Points/Game
1,Shaun Alexander,RB,Alabama,2000,72,218,4.58,0.00,0.00,0.00,0.00,0.00,Seattle Seahawks,1st,19th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327.00,1706.00,5.20,23.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,24.00,0.00,0.00,144.00,72,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.09,0.00,0.00,0.00,27.45,125.73,1.73,2.27,29.36,0.36,29.73,155.09,2.09,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,2.18,0.00,0.00,13.09
6,LaVar Arrington,OLB,Penn State,2000,75,250,4.53,0.00,0.00,0.00,0.00,0.00,Washington Redskins,1st,2nd pick,2000,12.00,0.00,0.00,0.00,0.00,0.00,1.00,27.00,27.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,0.00,0.00,0.00,0.00,0.00,72,3,0.00,0.00,0.00,0.00,0.00,0.08,2.25,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00
10,John Baker,P,North Texas,2000,75,227,0.00,0.00,0.00,0.00,0.00,0.00,0,Not Drafted,Not Drafted,Not Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,1.00,1.00,100.00,27.00,27.00,27.00,0.00,0.00,326.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72,3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.09,0.09,2.45,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
18,Anthony Becht,TE,West Virginia,2000,78,270,4.78,33.50,0.00,123.00,6.94,4.08,New York Jets,1st,27th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0

In [22]:
draft_data = draft_data[['PlayerName', 'Position', 'School', 'CombineYear', 'Height(in)', 'Weight', '40yd', 'Vertical',
                        'Bench', 'BroadJump', '3Cone', 'Shuttle', 'Draft Team', 'Round', 'Pick', 'Draft Year', 
                        'Defense_Games', 'Solo_Tackles', 'Assisted_Tackles', 'Ttl_Tackles', 'Loss', 'Sacks', 
                        'Defensive_Interceptions', 'Def_Int_Yds', 'Yds_per_Int', 'Pick_6', 'Defended_Passes',
                        'Recovered_Fumbles', 'Rec_Fumbles_Yds', 'Fumbles_Returned_TD', 'Forced_Fumbles', 
                        'Solo_Tackles/Game', 'Assisted_Tackles/Game', 'Ttl_Tackles/Game', 'Loss/Game', 'Sacks/Game', 
                        'Defensive_Interceptions/Game', 'Def_Int_Yds/Game', 'Pick_6/Game', 'Defended_Passes/Game',
                        'Recovered_Fumbles/Game', 'Rec_Fumbles_Yds/Game', 'Fumbles_Returned_TD/Game', 
                        'Forced_Fumbles/Game','Passing_Games', 'Completions', 'Pass_Attempts', 'Completion_Percent', 
                        'Pass_Yards', 'Pass_Yds_per_Attempt', 'Adj_Pass_Yds_per_Attempt', 'Pass_TDs', 
                        'Pass_Interceptions', 'Passer_Rating', 'Completions/Game', 'Pass_Attempts/Game', 
                        'Pass_Yards/Game', 'Pass_TDs/Game', 'Pass_Interceptions/Game', 'Rushing/Receiving_Games',
                        'Rush_Attempts', 'Rush_Yds', 'Rush_Yds_per_Attempt', 'Rush_TDs', 'Receptions', 'Rec_Yds',
                        'Rec_Yds_per_Reception', 'Rec_TDs', 'Plays_from_Scrimmage', 'Scrimmage_Yds',
                        'Scrimmage_Yds_per_Attempt', 'Scrimmage_TDs', 'Rush_Attempts/Game', 'Rush_Yds/Game', 
                        'Rush_TDs/Game', 'Receptions/Game', 'Rec_Yds/Game', 'Rec_TDs/Game', 
                        'Plays_from_Scrimmage/Game', 'Scrimmage_Yds/Game', 'Scrimmage_TDs/Game', 'Kicking_Games', 
                        'XP_Made', 'XP_Attempts', 'XP_Percent', 'FG_Made', 'FG_Attempts', 'FG_Percent',
                        'TTL_Kicking_Points', '#Punts', 'Punt_Yds', 'Yds_per_Punt', 'XP_Made/Game',
                        'XP_Attempts/Game', 'FG_Made/Game', 'FG_Attempts/Game', 'TTL_Kicking_Points/Game', 
                        '#Punts/Game', 'Punt_Yds/Game', 'Kickoff_Returns', 'KO_Return_Yds', 'Yds_per_KO_Return',
                        'KO_Returned_for_TD', 'Kickoff_Returns/Game', 'KO_Return_Yds/Game',
                        'KO_Returned_for_TD/Game', 'Scoring_Games', 'TD_Other', 'Ttl_TDs', '2PT_Conversions',
                        'Safety', 'TTL_Points', 'TD_Other/Game', 'Ttl_TDs/Game', '2PT_Conversions/Game',
                        'Safety/Game', 'TTL_Points/Game', 'Feet', 'Inches']]

draft_data.head()

,PlayerName,Position,School,CombineYear,Height(in),Weight,40yd,Vertical,Bench,BroadJump,3Cone,Shuttle,Draft Team,Round,Pick,Draft Year,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Solo_Tackles/Game,Assisted_Tackles/Game,Ttl_Tackles/Game,Loss/Game,Sacks/Game,Defensive_Interceptions/Game,Def_Int_Yds/Game,Pick_6/Game,Defended_Passes/Game,Recovered_Fumbles/Game,Rec_Fumbles_Yds/Game,Fumbles_Returned_TD/Game,Forced_Fumbles/Game,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,Completions/Game,Pass_Attempts/Game,Pass_Yards/Game,Pass_TDs/Game,Pass_Interceptions/Game,Rushing/Receiving_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Rush_Attempts/Game,Rush_Yds/Game,Rush_TDs/Game,Receptions/Game,Rec_Yds/Game,Rec_TDs/Game,Plays_from_Scrimmage/Game,Scrimmage_Yds/Game,Scrimmage_TDs/Game,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,#Punts,Punt_Yds,Yds_per_Punt,XP_Made/Game,XP_Attempts/Game,FG_Made/Game,FG_Attempts/Game,TTL_Kicking_Points/Game,#Punts/Game,Punt_Yds/Game,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Kickoff_Returns/Game,KO_Return_Yds/Game,KO_Returned_for_TD/Game,Scoring_Games,TD_Other,Ttl_TDs,2PT_Conversions,Safety,TTL_Points,TD_Other/Game,Ttl_TDs/Game,2PT_Conversions/Game,Safety/Game,TTL_Points/Game,Feet,Inches
1,Shaun Alexander,RB,Alabama,2000,72,218,4.58,0.00,0.00,0.00,0.00,0.00,Seattle Seahawks,1st,19th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327.00,1706.00,5.20,23.00,27.45,125.73,1.73,2.27,29.36,0.36,29.73,155.09,2.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,nan,nan,nan,11.00,0.00,24.00,0.00,0.00,144.00,0.00,2.18,0.00,0.00,13.09,72,0
6,LaVar Arrington,OLB,Penn State,2000,75,250,4.53,0.00,0.00,0.00,0.00,0.00,Washington Redskins,1st,2nd pick,2000,12.00,0.00,0.00,0.00,0.00,0.00,1.00,27.00,27.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.08,2.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,nan,nan,nan,12.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72,3
10,John Baker,P,North Texas,2000,75,227,0.00,0.00,0.00,0.00,0.00,0.00,0,Not Drafted,Not Drafted,Not Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,11.00,1.00,1.00,100.00,27.00,27.00,27.00,0.00,0.00,326.80,0.09,0.09,2.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,72,3
18,Anthony Becht,TE,West Virginia,2000,78,270,4.78,33.50,0.00,123.00,6.94,4.08,New York Jets,1st,27th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan

In [23]:
draft_data.fillna(0, inplace=True)
draft_data.head()

,PlayerName,Position,School,CombineYear,Height(in),Weight,40yd,Vertical,Bench,BroadJump,3Cone,Shuttle,Draft Team,Round,Pick,Draft Year,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Solo_Tackles/Game,Assisted_Tackles/Game,Ttl_Tackles/Game,Loss/Game,Sacks/Game,Defensive_Interceptions/Game,Def_Int_Yds/Game,Pick_6/Game,Defended_Passes/Game,Recovered_Fumbles/Game,Rec_Fumbles_Yds/Game,Fumbles_Returned_TD/Game,Forced_Fumbles/Game,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,Completions/Game,Pass_Attempts/Game,Pass_Yards/Game,Pass_TDs/Game,Pass_Interceptions/Game,Rushing/Receiving_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Rush_Attempts/Game,Rush_Yds/Game,Rush_TDs/Game,Receptions/Game,Rec_Yds/Game,Rec_TDs/Game,Plays_from_Scrimmage/Game,Scrimmage_Yds/Game,Scrimmage_TDs/Game,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,#Punts,Punt_Yds,Yds_per_Punt,XP_Made/Game,XP_Attempts/Game,FG_Made/Game,FG_Attempts/Game,TTL_Kicking_Points/Game,#Punts/Game,Punt_Yds/Game,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Kickoff_Returns/Game,KO_Return_Yds/Game,KO_Returned_for_TD/Game,Scoring_Games,TD_Other,Ttl_TDs,2PT_Conversions,Safety,TTL_Points,TD_Other/Game,Ttl_TDs/Game,2PT_Conversions/Game,Safety/Game,TTL_Points/Game,Feet,Inches
1,Shaun Alexander,RB,Alabama,2000,72,218,4.58,0.00,0.00,0.00,0.00,0.00,Seattle Seahawks,1st,19th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327.00,1706.00,5.20,23.00,27.45,125.73,1.73,2.27,29.36,0.36,29.73,155.09,2.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,24.00,0.00,0.00,144.00,0.00,2.18,0.00,0.00,13.09,72,0
6,LaVar Arrington,OLB,Penn State,2000,75,250,4.53,0.00,0.00,0.00,0.00,0.00,Washington Redskins,1st,2nd pick,2000,12.00,0.00,0.00,0.00,0.00,0.00,1.00,27.00,27.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.08,2.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72,3
10,John Baker,P,North Texas,2000,75,227,0.00,0.00,0.00,0.00,0.00,0.00,0,Not Drafted,Not Drafted,Not Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,1.00,1.00,100.00,27.00,27.00,27.00,0.00,0.00,326.80,0.09,0.09,2.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72,3
18,Anthony Becht,TE,West Virginia,2000,78,270,4.78,33.50,0.00,123.00,6.94,4.08,New York Jets,1st,27th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,

In [24]:
draft_data.shape

(5492, 119)

In [25]:
draft_data.reset_index(inplace=True)

In [27]:
draft_data.head()

,index,PlayerName,Position,School,CombineYear,Height(in),Weight,40yd,Vertical,Bench,BroadJump,3Cone,Shuttle,Draft Team,Round,Pick,Draft Year,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Solo_Tackles/Game,Assisted_Tackles/Game,Ttl_Tackles/Game,Loss/Game,Sacks/Game,Defensive_Interceptions/Game,Def_Int_Yds/Game,Pick_6/Game,Defended_Passes/Game,Recovered_Fumbles/Game,Rec_Fumbles_Yds/Game,Fumbles_Returned_TD/Game,Forced_Fumbles/Game,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,Completions/Game,Pass_Attempts/Game,Pass_Yards/Game,Pass_TDs/Game,Pass_Interceptions/Game,Rushing/Receiving_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Rush_Attempts/Game,Rush_Yds/Game,Rush_TDs/Game,Receptions/Game,Rec_Yds/Game,Rec_TDs/Game,Plays_from_Scrimmage/Game,Scrimmage_Yds/Game,Scrimmage_TDs/Game,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,#Punts,Punt_Yds,Yds_per_Punt,XP_Made/Game,XP_Attempts/Game,FG_Made/Game,FG_Attempts/Game,TTL_Kicking_Points/Game,#Punts/Game,Punt_Yds/Game,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Kickoff_Returns/Game,KO_Return_Yds/Game,KO_Returned_for_TD/Game,Scoring_Games,TD_Other,Ttl_TDs,2PT_Conversions,Safety,TTL_Points,TD_Other/Game,Ttl_TDs/Game,2PT_Conversions/Game,Safety/Game,TTL_Points/Game,Feet,Inches
0,1,Shaun Alexander,RB,Alabama,2000,72,218,4.58,0.00,0.00,0.00,0.00,0.00,Seattle Seahawks,1st,19th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327.00,1706.00,5.20,23.00,27.45,125.73,1.73,2.27,29.36,0.36,29.73,155.09,2.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,24.00,0.00,0.00,144.00,0.00,2.18,0.00,0.00,13.09,72,0
1,6,LaVar Arrington,OLB,Penn State,2000,75,250,4.53,0.00,0.00,0.00,0.00,0.00,Washington Redskins,1st,2nd pick,2000,12.00,0.00,0.00,0.00,0.00,0.00,1.00,27.00,27.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.08,2.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72,3
2,10,John Baker,P,North Texas,2000,75,227,0.00,0.00,0.00,0.00,0.00,0.00,0,Not Drafted,Not Drafted,Not Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,1.00,1.00,100.00,27.00,27.00,27.00,0.00,0.00,326.80,0.09,0.09,2.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72,3
3,18,Anthony Becht,TE,West Virginia,2000,78,270,4.78,33.50,0.00,123.00,6.94,4.08,New York Jets,1st,27th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0

In [28]:
draft_data.drop(columns=['index'], inplace=True)
draft_data.head()

,PlayerName,Position,School,CombineYear,Height(in),Weight,40yd,Vertical,Bench,BroadJump,3Cone,Shuttle,Draft Team,Round,Pick,Draft Year,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Solo_Tackles/Game,Assisted_Tackles/Game,Ttl_Tackles/Game,Loss/Game,Sacks/Game,Defensive_Interceptions/Game,Def_Int_Yds/Game,Pick_6/Game,Defended_Passes/Game,Recovered_Fumbles/Game,Rec_Fumbles_Yds/Game,Fumbles_Returned_TD/Game,Forced_Fumbles/Game,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,Completions/Game,Pass_Attempts/Game,Pass_Yards/Game,Pass_TDs/Game,Pass_Interceptions/Game,Rushing/Receiving_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Rush_Attempts/Game,Rush_Yds/Game,Rush_TDs/Game,Receptions/Game,Rec_Yds/Game,Rec_TDs/Game,Plays_from_Scrimmage/Game,Scrimmage_Yds/Game,Scrimmage_TDs/Game,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,#Punts,Punt_Yds,Yds_per_Punt,XP_Made/Game,XP_Attempts/Game,FG_Made/Game,FG_Attempts/Game,TTL_Kicking_Points/Game,#Punts/Game,Punt_Yds/Game,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Kickoff_Returns/Game,KO_Return_Yds/Game,KO_Returned_for_TD/Game,Scoring_Games,TD_Other,Ttl_TDs,2PT_Conversions,Safety,TTL_Points,TD_Other/Game,Ttl_TDs/Game,2PT_Conversions/Game,Safety/Game,TTL_Points/Game,Feet,Inches
0,Shaun Alexander,RB,Alabama,2000,72,218,4.58,0.00,0.00,0.00,0.00,0.00,Seattle Seahawks,1st,19th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327.00,1706.00,5.20,23.00,27.45,125.73,1.73,2.27,29.36,0.36,29.73,155.09,2.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,24.00,0.00,0.00,144.00,0.00,2.18,0.00,0.00,13.09,72,0
1,LaVar Arrington,OLB,Penn State,2000,75,250,4.53,0.00,0.00,0.00,0.00,0.00,Washington Redskins,1st,2nd pick,2000,12.00,0.00,0.00,0.00,0.00,0.00,1.00,27.00,27.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.08,2.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72,3
2,John Baker,P,North Texas,2000,75,227,0.00,0.00,0.00,0.00,0.00,0.00,0,Not Drafted,Not Drafted,Not Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,1.00,1.00,100.00,27.00,27.00,27.00,0.00,0.00,326.80,0.09,0.09,2.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72,3
3,Anthony Becht,TE,West Virginia,2000,78,270,4.78,33.50,0.00,123.00,6.94,4.08,New York Jets,1st,27th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.

In [29]:
### offensive line
Off_Line = ['C', 'OG', 'OT', 'OL']

### backs & receivers
Backs_Receivers = ['WR', 'TE', 'RB', 'FB']
Quarterbacks = ['QB']

### defensive line
Def_Line = ['DE', 'DT', 'DL', 'NT']

### linebackers
Linebackers = ['OLB', 'ILB', 'LB']

### defensive backs
Def_Backs = ['CB', 'FS', 'SS', 'S', 'DB', 'EDGE'] 

### special teams
Special_Teams = ['P', 'K', 'LS']

In [30]:
draft_data['Position'].head()

0     RB
1    OLB
2      P
3     TE
4     QB
Name: Position, dtype: object

In [31]:
if draft_data['Position'][0] in Backs_Receivers:
    print("I belong")
else:
    print('Nope')

Nope


In [34]:
if draft_data['Position'][1] in Backs_Receivers:
    print("I belong")
else:
    print('Nope')

Nope


In [40]:
draft_data['Pos'] = draft_data['Position']

for i in range(len(draft_data['Pos'])):
    item = draft_data['Pos'][i]
#     print(item)
    if item in Off_Line:
        draft_data['Pos'][i] = 'OffensiveLine'
    elif item in Backs_Receivers:
        draft_data['Pos'][i] = 'Receivers'
    elif item in Quarterbacks:
        draft_data['Pos'][i] = 'QBs'
    elif item in Linebackers:
        draft_data['Pos'][i] = 'LBs'
    elif item in Def_Backs:
        draft_data['Pos'][i] = 'DefensiveBacks'
    else:
        draft_data['Pos'][i] = 'SpecialTeams'

In [41]:
draft_data.head()

,PlayerName,Position,School,CombineYear,Height(in),Weight,40yd,Vertical,Bench,BroadJump,3Cone,Shuttle,Draft Team,Round,Pick,Draft Year,Defense_Games,Solo_Tackles,Assisted_Tackles,Ttl_Tackles,Loss,Sacks,Defensive_Interceptions,Def_Int_Yds,Yds_per_Int,Pick_6,Defended_Passes,Recovered_Fumbles,Rec_Fumbles_Yds,Fumbles_Returned_TD,Forced_Fumbles,Solo_Tackles/Game,Assisted_Tackles/Game,Ttl_Tackles/Game,Loss/Game,Sacks/Game,Defensive_Interceptions/Game,Def_Int_Yds/Game,Pick_6/Game,Defended_Passes/Game,Recovered_Fumbles/Game,Rec_Fumbles_Yds/Game,Fumbles_Returned_TD/Game,Forced_Fumbles/Game,Passing_Games,Completions,Pass_Attempts,Completion_Percent,Pass_Yards,Pass_Yds_per_Attempt,Adj_Pass_Yds_per_Attempt,Pass_TDs,Pass_Interceptions,Passer_Rating,Completions/Game,Pass_Attempts/Game,Pass_Yards/Game,Pass_TDs/Game,Pass_Interceptions/Game,Rushing/Receiving_Games,Rush_Attempts,Rush_Yds,Rush_Yds_per_Attempt,Rush_TDs,Receptions,Rec_Yds,Rec_Yds_per_Reception,Rec_TDs,Plays_from_Scrimmage,Scrimmage_Yds,Scrimmage_Yds_per_Attempt,Scrimmage_TDs,Rush_Attempts/Game,Rush_Yds/Game,Rush_TDs/Game,Receptions/Game,Rec_Yds/Game,Rec_TDs/Game,Plays_from_Scrimmage/Game,Scrimmage_Yds/Game,Scrimmage_TDs/Game,Kicking_Games,XP_Made,XP_Attempts,XP_Percent,FG_Made,FG_Attempts,FG_Percent,TTL_Kicking_Points,#Punts,Punt_Yds,Yds_per_Punt,XP_Made/Game,XP_Attempts/Game,FG_Made/Game,FG_Attempts/Game,TTL_Kicking_Points/Game,#Punts/Game,Punt_Yds/Game,Kickoff_Returns,KO_Return_Yds,Yds_per_KO_Return,KO_Returned_for_TD,Kickoff_Returns/Game,KO_Return_Yds/Game,KO_Returned_for_TD/Game,Scoring_Games,TD_Other,Ttl_TDs,2PT_Conversions,Safety,TTL_Points,TD_Other/Game,Ttl_TDs/Game,2PT_Conversions/Game,Safety/Game,TTL_Points/Game,Feet,Inches,Pos
0,Shaun Alexander,RB,Alabama,2000,72,218,4.58,0.00,0.00,0.00,0.00,0.00,Seattle Seahawks,1st,19th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,11.00,302.00,1383.00,4.60,19.00,25.00,323.00,12.90,4.00,327.00,1706.00,5.20,23.00,27.45,125.73,1.73,2.27,29.36,0.36,29.73,155.09,2.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,24.00,0.00,0.00,144.00,0.00,2.18,0.00,0.00,13.09,72,0,Receivers
1,LaVar Arrington,OLB,Penn State,2000,75,250,4.53,0.00,0.00,0.00,0.00,0.00,Washington Redskins,1st,2nd pick,2000,12.00,0.00,0.00,0.00,0.00,0.00,1.00,27.00,27.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.08,2.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72,3,LBs
2,John Baker,P,North Texas,2000,75,227,0.00,0.00,0.00,0.00,0.00,0.00,0,Not Drafted,Not Drafted,Not Drafted,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,1.00,1.00,100.00,27.00,27.00,27.00,0.00,0.00,326.80,0.09,0.09,2.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72,3,SpecialTeams
3,Anthony Becht,TE,West Virginia,2000,78,270,4.78,33.50,0.00,123.00,6.94,4.08,New York Jets,1st,27th pick,2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0

In [42]:
draft_data.to_pickle('cleaned_df')